In [1]:
import numpy
import pandas
import nltk
import sys
import mysecrets

<h1> Get positive and negative words from NRC Emotion Lexicon </h1>

In [22]:
def get_nrc_data(emotion):
    nrc = "/Users/rohankulkarni/nltk_data/NRC-Emotion-Lexicon/OneFilePerEmotion/"+ emotion +"-NRC-Emotion-Lexicon.txt" 

    
    f = open(nrc,'r')
    words = [word.split('\t')[0] for i, word in enumerate(f)]

    return words


In [23]:
pos = get_nrc_data("positive")
neg = get_nrc_data("negative")

<h1> Use Yelp API to get sample reviews </h1>

In [5]:
yelp_client_id = mysecrets.yelp_client_id
yelp_api_key = mysecrets.yelp_api_key

In [6]:
API_HOST = 'https://api.yelp.com' #The API url header
SEARCH_PATH = '/v3/businesses/search' #The path for an API request to find businesses
BUSINESS_PATH = '/v3/businesses/'  # The path to get data for a single business


In [35]:
def get_restaurants(api_key,location,number=15):
    import requests
    
     #Set up the search data dictionary. This contains the search parameters
    search_data = {
    'term': "restaurant",
    'location': location.replace(' ', '+'),
    'limit': number
    }
    
    #Create the API url
    url = API_HOST + SEARCH_PATH
    
    #The API Key data object
    headers = {
        'Authorization': 'Bearer %s' % api_key,
    }
    
    #Get the response object and convert from JSON into a python list
    response = requests.request('GET', url, headers=headers, params=search_data).json()
    #print(response)
    
    #Extract the businesses from the response object
    businesses = response.get('businesses')
    
    #Extract the business id and business name for each business into a list
    return_data = [(business['id'],business['name']) for business in businesses]
    return return_data

get_restaurants(yelp_api_key,"Princeton",15)

#Let's choose the restaurant: Tacoria mC0Oir4IMjMSzJGORIW6pw

[('ANG7pXCpE1iPzDSsB-abwg', 'The Meeting House'),
 ('SgXxGCzsf5xhEo5mYpBHEw', 'Agricola Eatery'),
 ('ytvXEa1BdgdMQPOGANLBCQ', 'The Dinky Bar & Kitchen'),
 ('aA3pwxdS76TCPskajGj_Qg', 'Mistral'),
 ('3g-OXwOHASK8bCuInOfNKg', 'Blue Point Grill'),
 ('K08kXfrLkM6A7Zixwb5ljQ', 'Roots Ocean Prime'),
 ('wFXwhiQ3Hp2SsJb-iNG4BA', 'Elements'),
 ('h27I0U_S4TPm3zwvZtbnJA', 'Laurea Restaurant'),
 ('0YkXhcfpFHuYw_33Iq9kgQ', 'The Perch At Peacock Inn'),
 ('LxL23QJGTuiNW_lfE1mn8Q', 'Local Greek'),
 ('4CXO92H2UsgSI4koZNTBQg', 'Salt Creek Grille'),
 ('zdj5iGqtqJmLBL1UFsa5mA', 'Lan Ramen'),
 ('rXy_19w4bAoyGImmZ4w23Q', 'Kristines'),
 ('mC0Oir4IMjMSzJGORIW6pw', 'Tacoria'),
 ('Q0hNFisTc8hm5t1sCVHB4A', 'Schouse')]

In [36]:
taco = "mC0Oir4IMjMSzJGORIW6pw"

In [37]:
def get_business_review(api_key,business_id):
    import requests
    
    #API path
    business_path = BUSINESS_PATH + business_id+"/reviews"
    
    #API url
    url = API_HOST + business_path

    #API Key data object
    headers = {
        'Authorization': 'Bearer %s' % api_key,
    }

    #Get the response and convert from json into a python object
    response = requests.request('GET', url, headers=headers).json()
    #print(response)
    
    #Yelp gives review snippets of a few reviews, concatenate these into a single string
    review_text = ''
    for review in response['reviews']:
        review_text += review['text']
    return review_text

get_business_review(yelp_api_key,taco)

"Fresh ingredients and a flavorful adventure is how I describe Tacoria.  \nThe staff is always helpful and patient.  \nI usually take out.What a disappointment! Are of the people there on downers? They all seem to have flatlined. No emotion, no affect. It's not like they could be tired from...Burritos here are sheer perfection. Stuffed with tasty meats, fresh veggies, cheese, and rice. It's bursting with flavor.  The warm toasted tortilla just..."

In [11]:
import os

import openai
openai.api_key = mysecrets.key

def get_res(p):
    obj = openai.Completion.create(
            model="text-davinci-003",
            prompt=p,
            temperature=0.1,
            max_tokens=1000
        )
    
    return obj["choices"][0]["text"].replace("\n", "")


In [15]:
prompt = "Make a positive review. Use the word: delicious"
get_res(prompt)

'I had the most delicious meal at this restaurant! The food was cooked to perfection and the flavors were amazing. I would highly recommend this place to anyone looking for a great dining experience.'

In [92]:
#sample prompt

def generate_prompt(animal):
    return """Suggest three names for an animal that is a superhero.

Animal: Cat
Names: Captain Sharpclaw, Agent Fluffball, The Incredible Feline
Animal: Dog
Names: Ruff the Protector, Wonder Canine, Sir Barks-a-Lot
Animal: {}
Names:""".format(
        animal.capitalize()
    )

In [93]:
prompt = generate_prompt("cow")
get_res(prompt)

' Mighty Moo, Super Bovine, Udderly Amazing'